# Modelado en Optimización (IIND-2501)

## Lección 3.3: Optimización lineal (formato estándar, holguras y bases)

El **propósito** de esta lección es construir intuición geométrica de:
- Región factible, restricciones activas/inactivas, holguras.
- Variables básicas y no básicas (en $\mathbb{R}^n$, los vértices son intersección de $n$ o más restricciones activas).
- Conexión entre formato canónico, estándar y matricial.

In [1]:
'''
Importaciones y configuraciones generales del notebook. No modificar.
'''
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math
from ipywidgets import FloatSlider, VBox, HBox, HTML, interactive_output, Checkbox, Button, Output, Layout

from helpers_bases import plot_method_graph, plot_LP, eval_bases, slacks, bar_slacks, mat_to_bmatrix, vec_to_bmatrix, basis_info

### Representación general de un problema de optimización

Un problema de optimización con $n$ variables y $m$ restricciones puede escribirse de manera general como:

$$\max\; c^\top x$$
$$\text{s.a.}$$
$$A x \le b,\;$$
$$x\ge 0,\;$$
$$x\in\mathbb{R}^2.$$

En esta **representación matricial**, $A\in\mathbb{R}^{m\times n}$ contiene los coeficientes de las variables en las restricciones, $b\in\mathbb{R}^{m}$ contiene los términos independientes (lado derecho) de las restricciones, y $c\in\mathbb{R}^{n}$ contiene los costos asociados a las variables en la función objetivo (ver diapositivas).

#### Ejemplo:

$$\max\ z = 3x_1 + 2x_2$$

Sujeto a
$$
\begin{aligned}
x_1 + x_2 &\le 6 \\
x_1 + 2 x_2 &\le 8 \\
2 x_2 + x_2 &\le 8 \\
x_1, x_2 &\ge 0 \, .
\end{aligned}
$$

En notación matricial:
$$
A =
\begin{bmatrix}
1 & 1 \\
1 & 2 \\
2 & 1
\end{bmatrix},
\quad
b =
\begin{bmatrix}
6 \\ 8 \\ 8
\end{bmatrix},
\quad
c =
\begin{bmatrix}
3 \\ 2
\end{bmatrix} .
$$

## Formatos de representación 

- **Formato canónico**: implica expresar problemas de maximización con restricciones `<=`(y de minimización con restricciones `>=`); el ejemplo anterior ya está dado en formato canónico.
  
- **Formato estándar**: implica expresar el problema solo con restricciones de igualdad para resolver sistemas de ecuaciones exactos. Para esto, introduce `variables de holgura` ($s$) que absorben "lo que le falta" a una restricción para cumplirse en igualdad, reemplazando las restricciones originales por:
$$\max\; c^\top x$$
$$A x + s = b,$$
$$x\ge 0,$$
$$s\ge 0.$$

Para el caso del ejemplo anterior, la representación en **formato estándar** sería:
$$\max\ z = 3x_1 + 2x_2$$
$$
\begin{aligned}
x_1 + x_2 + s_1 &= 6 \\
x_1 + 2 x_2 + s_2 &= 8 \\
2 x_2 + x_2 + s_3 &= 8 \\
x_1, x_2, s_1, s_2, s_3 &\ge 0 \, .
\end{aligned}
$$

De forma que:

$$
A =
\begin{bmatrix}
1 & 1 & 1 & 0 & 0 \\
1 & 2 & 0 & 1 & 0\\
2 & 1 & 0 & 0 & 1
\end{bmatrix},
\quad
b =
\begin{bmatrix}
6 \\ 8 \\ 8
\end{bmatrix},
\quad
c =
\begin{bmatrix}
3 \\ 2 \\ 0 \\ 0 \\ 0
\end{bmatrix} .
$$

Observa que:
$$
\begin{aligned}
s_1 &= 6 - (x_1 + x_2) \\
s_2 &= 8 - (x_1 + 2 x_2 ) \\
s_3 &= 8 - (2 x_2 + x_2)\\
\end{aligned}
$$

In [2]:
A = np.array([[1.,1.],[1.,2.],[2.,1.]], float)
b = np.array([6.,8.,8.], float)
c = np.array([3.,2.], float)

## Conceptos de *holgura y restricción activa*

Utiliza el siguiente *recurso interactivo* para evaluar diferentes puntos $(x_1, x_2)$, verificar factibilidad, y comprender el papel de las variables de holgura.

- Desliza `x1` y `x2` para **probar factibilidad** $(Ax \le b$).
- El vector de **holguras** es $s = b - A x$.
  - Si $s_i > 0$: la restricción $i$ está **inactiva**.
  - Si $s_i = 0$: la restricción $i$ está **activa**.
  - Si $s_i < 0$: la restricción $i$ se está incumpliendo.
- El gráfico 2D muestra las restricciones (rectas $a_i^\top x = b_i$), la región factible y el punto $(x_1, x_2)$ (todo en el espacio de $x_1,x_2$).
- El gráfico de barras horizontales muestra los valores de **todas las variables**: $[x_1, x_2, s_1, s_2, s_3]$.

In [3]:
plot_LP(A,b,c)

Introducir noción de **punto extremo**.

# Cómo solucionar el sistema de ecuaciones

Al introducir **holguras**, obtenemos un vector de decisión ampliado $x = [\,x_1,\ x_2,\ s_1,\ s_2,\ s_3\,]^\top$ y matrices:

$$
A_{\text{full}} =
\begin{bmatrix}
1 & 1 & 1 & 0 & 0 \\
1 & 2 & 0 & 1 & 0 \\
2 & 1 & 0 & 0 & 1
\end{bmatrix},
\quad
b =
\begin{bmatrix}
6 \\ 8 \\ 8
\end{bmatrix},
\quad
c_{\text{full}} =
\begin{bmatrix}
3 \\ 2 \\ 0 \\ 0 \\ 0
\end{bmatrix} .
$$

Un sistema de ecuaciones de 3x5 ($m\times n$) tiene infitas soluciones. Por eso, fijamos $n-m$ variables en cero para obtener un sistema de $m\times m$, *con igual número de ecuaciones e incógnitas*. Elegimos $m$ columnas (con $m=3$) como **base** $B$. Si $B$ es invertible, podemos encontrar el valor de las variables que no fijamos en cero:

$$x_N = 0,\qquad x_B = B^{-1} b.$$

# Bases válidas — selección por **checkboxes**
Selecciona exactamente **m** variables para formar una **base** $B$ candidata. Fijamos las demás variables en cero (**no básicas**). Si la submatriz es invertible, resolvemos $B x_B = b$, reportamos soluciones, y graficamos el punto resultante en el espacio de $(x_1, x_2)$.

In [4]:
A_full = np.array([[1,1,1,0,0],
                   [1,2,0,1,0],
                   [2,1,0,0,1]], float)

var_names=['x1','x2','s1','s2','s3']


In [5]:
checks = [Checkbox(value=(name in ['s1','s2','s3']), description=name, indent=False, layout=Layout(width='70px')) for name in var_names]
btn_eval = Button(description="Evaluar base", button_style="primary")
msg = HTML(); out_math = Output(); out_plot = Output()
def on_eval(_):
    out_math.clear_output(); out_plot.clear_output(); msg.value=""
    Bcols = [i for i,cb in enumerate(checks) if cb.value]
    if len(Bcols)!=3: msg.value="<b>Selecciona exactamente m=3.</b>"; return
    info = basis_info(A_full, b, np.r_[c,0,0,0], Bcols)
    x = info['x']
    with out_math:
        display(Math(r"I_B=\{"+", ".join(str(i+1) for i in info['Bcols'])+r"\},\ I_N=\{"+", ".join(str(j+1) for j in info['Ncols'])+r"\}"))
        display(Math(r"\textbf{B}="+mat_to_bmatrix(info['B'])+r"\textbf{N}="+mat_to_bmatrix(info['N'])))        
        display(Math(r"\mathbf{x}_B=\mathbf{B}^{-1}\mathbf{b}="+vec_to_bmatrix(info['xB'], fmt='{:.3g}')+r"; \mathbf{x}_N=\mathbf{0},\ " + r"\Rightarrow\ \mathbf{x}="+vec_to_bmatrix(x, fmt='{:.3g}') ))
    with out_plot:
        fig, (ax1, ax2) = plt.subplots(1,2, figsize=(11,5))
        plot_method_graph(A,b,x=x[:2],ax=ax1,title="Region factible y vértice de la base elegida", show_fill=True)
        s = slacks(A,b,x[:2]); 
        colors=['tab:grey','tab:grey','tab:blue','tab:orange','tab:green']        
        bar_slacks(np.hstack([x[:2], s]), colors=colors, ax=ax2, names=["x1","x2","s1","s2","s3"])
        plt.show()
btn_eval.on_click(on_eval)
display(VBox([HBox(checks), btn_eval, msg, out_math, out_plot]))